In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import path

dogs_intakes_df = pd.read_csv("center_data\\Austin_Animal_Center_Intakes.csv")
dogs_intakes_df = dogs_intakes_df.loc[dogs_intakes_df['Animal Type'] == 'Dog']
dogs_intakes_df.sort_values(by=['MonthYear', 'Animal ID'])

dogs_intakes_df = dogs_intakes_df.rename(columns = {
    "MonthYear":"MonthYear_intake"
})

dogs_intakes_df["MonthYear_intake"] = pd.to_datetime(dogs_intakes_df['MonthYear_intake']).dt.normalize()


# dogs_intakes_df.to_csv('dog_intakes.csv')
dogs_intakes_df

,Animal ID,Name,DateTime,MonthYear_intake,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A786884,*Brock,01/03/2019 04:19:00 PM,2019-01-03,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor
1,A706918,Belle,07/05/2015 12:59:00 PM,2015-07-05,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver
2,A724273,Runster,04/14/2016 06:43:00 PM,2016-04-14,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White
4,A682524,Rio,06/29/2014 10:38:00 AM,2014-06-29,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray
5,A743852,Odin,02/18/2017 12:46:00 PM,2017-02-18,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,2 years,Labrador Retriever Mix,Chocolate
...,...,...,...,...,...,...,...,...,...,...,...,...
113781,A804034,Lola,11/26/2019 06:20:00 PM,2019-11-26,Austin (TX),Owner Surrender,Normal,Dog,Intact Female,6 months,German Shepherd Mix,Black/Tan
113783,A804034,Lola,09/09/2019 06:49:00 PM,2019-09-09,4602 E Stassney Lane in Austin (TX),Stray,Normal,Dog,Intact Female,3 months,German Shepherd Mix,Black/Tan
113784,A804932,Nina,09/20/2019 05:33:00 PM,2019-09-20,1720 Woodward Street in Austin (TX),Stray,Normal,Dog,Intact Female,2 years,Cardigan Welsh Corgi Mix,Tan
113785,A804932,Nina,11/24/2019 03:23:00 PM,2019-11-24,Austin (TX),Owner Surrender,Normal,Dog,Intact Female,2 years,Cardigan Welsh Corgi Mix,Tan


In [30]:
outcomes_df = pd.read_csv("center_data\\Austin_Animal_Center_Outcomes.csv")

#Find only those dogs that resulted in adoption.
dogs_outcomes_df = outcomes_df[(outcomes_df['Animal Type'] == 'Dog') & (outcomes_df['Outcome Type'] == 'Adoption')]

dogs_outcomes_df = dogs_outcomes_df.rename(columns = {
    "MonthYear":"MonthYear_outcome"
})

dogs_outcomes_df["MonthYear_outcome"] = pd.to_datetime(dogs_outcomes_df["MonthYear_outcome"]).dt.normalize()

dogs_outcomes_df


,Animal ID,Name,DateTime,MonthYear_outcome,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A789027,Lennie,02/17/2019 11:44:00 AM,2019-02-17,02/13/2017,Adoption,NaN,Dog,Neutered Male,2 years,Chihuahua Shorthair Mix,Cream
1,A720371,Moose,02/13/2016 05:59:00 PM,2016-02-13,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
9,A789027,Lennie,03/10/2019 12:25:00 PM,2019-03-10,02/13/2017,Adoption,NaN,Dog,Neutered Male,2 years,Chihuahua Shorthair Mix,Cream
17,A794494,Zoey,05/14/2019 04:27:00 PM,2019-05-14,06/14/2018,Adoption,NaN,Dog,Spayed Female,10 months,Wire Hair Fox Terrier Mix,White/Black
20,A764361,Astro,01/03/2018 07:15:00 PM,2018-01-03,12/28/2016,Adoption,NaN,Dog,Neutered Male,1 year,Norwich Terrier Mix,Black/Tan
...,...,...,...,...,...,...,...,...,...,...,...,...
113818,A788859,Tobias,02/16/2019 04:45:00 PM,2019-02-16,12/24/2017,Adoption,NaN,Dog,Neutered Male,1 year,Lhasa Apso/Standard Poodle,Gold/Tan
113822,A774560,Jake,07/29/2018 01:55:00 PM,2018-07-29,04/17/2018,Adoption,NaN,Dog,Neutered Male,3 months,Rat Terrier Mix,Tricolor
113829,A758344,*Zoe,12/10/2017 12:28:00 PM,2017-12-10,09/14/2015,Adoption,NaN,Dog,Spayed Female,2 years,Pit Bull Mix,White/Chocolate
113832,A780375,Finnegan,09/17/2018 05:27:00 PM,2018-09-17,03/13/2018,Adoption,NaN,Dog,Neutered Male,6 months,Rat Terrier Mix,White/Black


In [44]:
dog_concat_df = \
    pd\
    .concat([dogs_intakes_df, dogs_outcomes_df], sort=False)\
    .sort_values(by=["Animal ID", "MonthYear_intake", "MonthYear_outcome"])
    
# dog_concat_df = dog_concat_df[dog_concat_df["Animal ID"] == "A754989"]
dog_concat_df = dog_concat_df[dog_concat_df["MonthYear_intake"] == "2020-01-07"]
dog_concat_df

,Animal ID,Name,DateTime,MonthYear_intake,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color,MonthYear_outcome,Date of Birth,Outcome Type,Outcome Subtype,Sex upon Outcome,Age upon Outcome
111536,A584775,Prince,01/07/2020 03:03:00 PM,2020-01-07,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,10 years,Pit Bull,White/Blue Merle,NaT,NaN,NaN,NaN,NaN,NaN
111435,A643015,Mojo,01/07/2020 03:03:00 PM,2020-01-07,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,11 years,Boxer,Brown/White,NaT,NaN,NaN,NaN,NaN,NaN
111763,A724529,Rosita,01/07/2020 03:36:00 PM,2020-01-07,2953 Bridekirk Drive in Round Rock (TX),Stray,Normal,Dog,Intact Female,4 years,Chihuahua Shorthair Mix,Black/Brown,NaT,NaN,NaN,NaN,NaN,NaN
111410,A734273,Olive,01/07/2020 12:21:00 PM,2020-01-07,13701 Sea Biscuit Drive in Travis (TX),Stray,Normal,Dog,Spayed Female,8 years,Chihuahua Shorthair Mix,Tan,NaT,NaN,NaN,NaN,NaN,NaN
111433,A789702,*Himalaya,01/07/2020 11:34:00 AM,2020-01-07,Austin (TX),Owner Surrender,Normal,Dog,Spayed Female,1 year,Pit Bull Mix,White/Tan,NaT,NaN,NaN,NaN,NaN,NaN
111167,A802052,*Baxter,01/07/2020 08:37:00 AM,2020-01-07,Williamson (TX),Owner Surrender,Normal,Dog,Neutered Male,7 years,Beagle/Basset Hound,Brown/White,NaT,NaN,NaN,NaN,NaN,NaN
111428,A811400,Lucy,01/07/2020 01:59:00 PM,2020-01-07,Clawson Road And Grayford Drive in Austin (TX),Stray,Normal,Dog,Intact Female,10 years,Pug,Black/White,NaT,NaN,NaN,NaN,NaN,NaN
111164,A811634,NaN,01/07/2020 07:41:00 AM,2020-01-07,8415 Research in Austin (TX),Stray,Injured,Dog,Intact Female,2 years,Pit Bull,Brown Brindle,NaT,NaN,NaN,NaN,NaN,NaN
111430,A811635,Pal,01/07/2020 08:17:00 AM,2020-01-07,401 W Braker Lane in Austin (TX),Stray,Injured,Dog,Neutered Male,14 years,Collie Rough,Brown/White,NaT,NaN,NaN,NaN,NaN,NaN
111170,A811643,Cocoa,01/07/2020 10:28:00 AM,2020-01-07,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,4 years,Chihuahua Shorthair,Chocolate/White,NaT,NaN,NaN,NaN,NaN,NaN


In [4]:
dog_df_merged = pd.merge(dogs_outcomes_df, dogs_intakes_df[['Animal ID', 
    'Name', 'DateTime', 'MonthYear', 'Found Location',
    'Intake Type', 'Intake Condition', 'Animal Type', 'Sex upon Intake',
    'Age upon Intake', 'Breed', 'Color']], 
    on='Animal ID', 
    how='left', 
    suffixes=('_outcome', '_intake'))


print(dog_df_merged.head())

print( dog_df_merged.mode()['Animal ID'][0])



#Normalize cuts off the time of the datetime object by converting it to midnight.
dog_df_merged['Date_outcome'] = pd.to_datetime(dog_df_merged['MonthYear_outcome']).dt.normalize()

dog_df_merged['Date_intake'] =  pd.to_datetime(dog_df_merged['MonthYear_intake']).dt.normalize()

dog_df_merged['stay_length']=abs((dog_df_merged['Date_outcome'])-\
    (dog_df_merged['Date_intake']))

dog_df_merged = dog_df_merged.sort_values(by=[ 'Animal ID', 'MonthYear_intake'])

print(dog_df_merged.shape)

dog_df_merged.head(30)

  Animal ID Name_outcome        DateTime_outcome       MonthYear_outcome  \
0   A666770         Josh  01/01/2014 01:32:00 PM  01/01/2014 01:32:00 PM   
1   A666770         Josh  01/01/2014 01:32:00 PM  01/01/2014 01:32:00 PM   
2   A643402       Scuffy  01/01/2014 02:38:00 PM  01/01/2014 02:38:00 PM   
3   A662850      Woofers  01/01/2014 03:17:00 PM  01/01/2014 03:17:00 PM   
4   A516660       Marley  01/01/2014 03:32:00 PM  01/01/2014 03:32:00 PM   

  Date of Birth Outcome Type Outcome Subtype Animal Type_outcome  \
0    11/07/2010     Adoption             NaN                 Dog   
1    11/07/2010     Adoption             NaN                 Dog   
2    10/01/2012     Adoption             NaN                 Dog   
3    09/09/2008     Adoption             NaN                 Dog   
4    07/21/2007     Adoption             NaN                 Dog   

  Sex upon Outcome Age upon Outcome  ... DateTime_intake MonthYear_intake  \
0    Neutered Male          3 years  ...    11/7/13 7:46 

,Animal ID,Name_outcome,DateTime_outcome,MonthYear_outcome,Date of Birth,Outcome Type,Outcome Subtype,Animal Type_outcome,Sex upon Outcome,Age upon Outcome,...,Intake Type,Intake Condition,Animal Type_intake,Sex upon Intake,Age upon Intake,Breed_intake,Color_intake,Date_outcome,Date_intake,stay_length
37617,A200922,Carlos,11/22/2013 09:44:00 AM,11/22/2013 09:44:00 AM,10/03/1997,Adoption,Foster,Dog,Neutered Male,16 years,...,Owner Surrender,Normal,Dog,Neutered Male,16 years,Dachshund Mix,Black/Tan,2013-11-22,2013-10-03,50 days
32498,A210457,Caleb,10/07/2016 12:34:00 PM,10/07/2016 12:34:00 PM,06/01/1999,Adoption,Foster,Dog,Neutered Male,17 years,...,Public Assist,Aged,Dog,Neutered Male,17 years,Chihuahua Shorthair,Tan/Black,2016-10-07,2016-09-28,9 days
35198,A226069,Cedar,10/29/2015 03:00:00 PM,10/29/2015 03:00:00 PM,06/17/2000,Adoption,Foster,Dog,Neutered Male,15 years,...,Stray,Normal,Dog,Neutered Male,15 years,Labrador Retriever/Beagle,Sable/White,2015-10-29,2015-10-06,23 days
37005,A249087,*Polly,11/15/2016 10:37:00 AM,11/15/2016 10:37:00 AM,01/02/2001,Adoption,Foster,Dog,Spayed Female,15 years,...,Stray,Normal,Dog,Spayed Female,15 years,German Shepherd Mix,Black/Brown,2016-11-15,2016-08-17,90 days
37043,A274546,Sophie,11/16/2013 12:24:00 PM,11/16/2013 12:24:00 PM,05/25/2002,Adoption,NaN,Dog,Spayed Female,11 years,...,Stray,Normal,Dog,Spayed Female,11 years,Pointer Mix,Tricolor,2013-11-16,2013-11-13,3 days
15582,A281542,George,05/19/2014 12:32:00 PM,05/19/2014 12:32:00 PM,09/25/2001,Adoption,Foster,Dog,Neutered Male,12 years,...,Owner Surrender,Normal,Dog,Neutered Male,12 years,Labrador Retriever Mix,Black/Brown Brindle,2014-05-19,2013-12-23,147 days
23663,A302820,*Ladybug,07/25/2014 10:30:00 AM,07/25/2014 10:30:00 AM,03/19/2003,Adoption,Foster,Dog,Spayed Female,11 years,...,Owner Surrender,Normal,Dog,Spayed Female,10 years,German Shepherd Mix,Brown,2014-07-25,2014-02-27,148 days
9798,A338231,Boots,03/25/2018 01:25:00 PM,03/25/2018 01:25:00 PM,02/28/2002,Adoption,NaN,Dog,Neutered Male,16 years,...,Owner Surrender,Normal,Dog,Neutered Male,16 years,Border Collie Mix,Black/White,2018-03-25,2018-02-25,28 days
9799,A338231,Boots,03/25/2018 01:25:00 PM,03/25/2018 01:25:00 PM,02/28/2002,Adoption,NaN,Dog,Neutered Male,16 years,...,Stray,Normal,Dog,Neutered Male,13 years,Border Collie Mix,Black/White,2018-03-25,2015-05-18,1042 days
8741,A338643,Herbie,03/16/2017 06:55:00 PM,03/16/2017 06:55:00 PM,01/24/2004,Adoption,Foster,Dog,Neutered Male,13 years,...,Owner Surrender,Normal,Dog,Neutered Male,12 years,Labrador Retriever/Australian Cattle Dog,Black/White,2017-03-16,2016-12-16,90 days


In [ ]:
total_dogs = pd.concat()

In [5]:
dog_df_merged.loc[dog_df_merged['Animal ID'] == 'A754989']

,Animal ID,Name_outcome,DateTime_outcome,MonthYear_outcome,Date of Birth,Outcome Type,Outcome Subtype,Animal Type_outcome,Sex upon Outcome,Age upon Outcome,...,Intake Type,Intake Condition,Animal Type_intake,Sex upon Intake,Age upon Intake,Breed_intake,Color_intake,Date_outcome,Date_intake,stay_length
2020,A754989,Molly,01/18/2019 07:17:00 PM,01/18/2019 07:17:00 PM,05/27/2017,Adoption,NaN,Dog,Spayed Female,1 year,...,Owner Surrender,Normal,Dog,Spayed Female,1 year,Labrador Retriever Mix,Tan,2019-01-18,2019-01-19,1 days
2104,A754989,Molly,01/19/2019 01:40:00 PM,01/19/2019 01:40:00 PM,05/27/2017,Adoption,NaN,Dog,Spayed Female,1 year,...,Owner Surrender,Normal,Dog,Spayed Female,1 year,Labrador Retriever Mix,Tan,2019-01-19,2019-01-19,0 days
5952,A754989,Molly,02/19/2019 07:04:00 PM,02/19/2019 07:04:00 PM,05/27/2017,Adoption,NaN,Dog,Spayed Female,1 year,...,Owner Surrender,Normal,Dog,Spayed Female,1 year,Labrador Retriever Mix,Tan,2019-02-19,2019-01-19,31 days
24197,A754989,Molly,07/29/2017 06:39:00 PM,07/29/2017 06:39:00 PM,05/27/2017,Adoption,NaN,Dog,Spayed Female,2 months,...,Owner Surrender,Normal,Dog,Spayed Female,1 year,Labrador Retriever Mix,Tan,2017-07-29,2019-01-19,539 days
29332,A754989,Molly,09/08/2019 02:43:00 PM,09/08/2019 02:43:00 PM,05/27/2017,Adoption,NaN,Dog,Spayed Female,2 years,...,Owner Surrender,Normal,Dog,Spayed Female,1 year,Labrador Retriever Mix,Tan,2019-09-08,2019-01-19,232 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24204,A754989,Molly,07/29/2017 06:39:00 PM,07/29/2017 06:39:00 PM,05/27/2017,Adoption,NaN,Dog,Spayed Female,2 months,...,Owner Surrender,Normal,Dog,Spayed Female,2 years,Labrador Retriever Mix,Tan,2017-07-29,2019-09-04,767 days
29339,A754989,Molly,09/08/2019 02:43:00 PM,09/08/2019 02:43:00 PM,05/27/2017,Adoption,NaN,Dog,Spayed Female,2 years,...,Owner Surrender,Normal,Dog,Spayed Female,2 years,Labrador Retriever Mix,Tan,2019-09-08,2019-09-04,4 days
30569,A754989,Molly,09/19/2019 05:46:00 PM,09/19/2019 05:46:00 PM,05/27/2017,Adoption,NaN,Dog,Spayed Female,2 years,...,Owner Surrender,Normal,Dog,Spayed Female,2 years,Labrador Retriever Mix,Tan,2019-09-19,2019-09-04,15 days
34898,A754989,Molly,10/26/2019 04:06:00 PM,10/26/2019 04:06:00 PM,05/27/2017,Adoption,NaN,Dog,Spayed Female,2 years,...,Owner Surrender,Normal,Dog,Spayed Female,2 years,Labrador Retriever Mix,Tan,2019-10-26,2019-09-04,52 days


In [ ]:
# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata
import requests

app_token="3mubvsyw1rybaisquwqluss7zqf6c3nqxu5g4x56odov7ggexx"
def get_soda_api_data(endpoint, app_token, count=1000, offset=0, return_df=True):
    params = {'$$app_token': app_token, '$limit': count, '$offset': offset, }
    
    results = []

    while True:

        try:
            r = requests.get(endpoint, params=params)
            rcontent = r.json()

            if rcontent == []:
                break

            results.append(rcontent)
            offset += count 
            params['$offset'] = offset

        except HTTPError as err:

            if err.response.status_code == '404':
                break
            else:
                print(err.response.status_code)
    
    if return_df:
        results_df = pd.DataFrame()

        for i in results:
            results_df = results_df.append(pd.io.json.json_normalize(i))
        
        return results_df
    
    else:
        return results
    
endpoint="https://data.austintexas.gov/resource/wter-evkm.json"

get_soda_api_data(endpoint, app_token)